In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, cycle, product, islice, chain
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional
from sys         import maxsize

import re
import ast
import operator

import numpy as np

In [2]:
def read_data(input: str, parser=str, sep='\n', testing=False) -> list:
    if testing:
        sections = input.split(sep)
    else:
        sections = open(input).read().split(sep)
    return [parser(section) for section in sections]

In [3]:
test_string = """class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12"""

In [4]:
class Ranges(tuple):
    '''A list of ranges, supports `in`'''
    def __contains__(self, value):
        return any(value in item for item in self)

def parse_fields(input: str):
    field, start1, end1, start2, end2 = re.match(r"([ \w]+): (\d+)-(\d+) or (\d+)-(\d+)", input).groups()
    return field, Ranges((range(int(start1), int(end1) + 1), range(int(start2), int(end2) + 1)))

def parse_ticket(input: str):
    return [int(x) for x in input.split(",")]

def parse_all(input: List[str]):
    fields_raw, my_ticket_raw, nearby_raw = input
    fields = dict([parse_fields(fields) for fields in fields_raw.split("\n")])
    my_ticket = parse_ticket(my_ticket_raw.split("\n")[-1])
    nearby_tickets = [parse_ticket(tik) for tik in nearby_raw.split("\n")[1:]]
    return fields, my_ticket, nearby_tickets

def run_part1(input: List[str]):
    fields, my_ticket, nearby_tickets = parse_all(input)
    all_ranges = Ranges(fields.values())
    return sum([field for ticket in nearby_tickets 
            for field in ticket 
            if field not in all_ranges])

Part I  

Consider the validity of the nearby tickets you scanned. What is your ticket scanning error rate?

In [5]:
test_ins = read_data(test_string, sep='\n\n', testing=True)
run_part1(test_ins)

71

In [6]:
real_ins = read_data("input.txt", sep='\n\n')
run_part1(real_ins)

25916

Part II

Once you work out which field is which, look for the six fields on your ticket that start with the word departure. What do you get if you multiply those six values together?

In [7]:
def get_invalid_fields(entries, fields):
    return [name for name in fields
            if any(entry not in fields[name] for entry in entries)]

def remove_others(position, possible_fields):
    for field in possible_fields:
        if position != field:
            possible_fields[field] -= possible_fields[position] 

def valid_ticket(ticket, all_ranges) -> bool:
    return all(field in all_ranges for field in ticket)

def decode_fields(input: List[str]):
    fields, my_ticket, nearby_tickets = parse_all(input)
    all_ranges = Ranges(fields.values())

    import numpy as np
    all_tickets = [my_ticket] + nearby_tickets
    valid_tickets = np.array([ticket for ticket in all_tickets if valid_ticket(ticket, all_ranges)])

    # initializes possible fields for each ticket entry position
    possible_fields = {i: set(fields) for i in range(len(my_ticket))}
    
    # as we iterate, the number of fields in possible_fields will be iteratively removed
    while any(len(possible_fields[field]) > 1 for field in possible_fields):
        for position in possible_fields:
            position_entries = valid_tickets[:, position]
            
            # update the invalid fields
            invalid_fields = set(get_invalid_fields(position_entries, fields))
            
            # remove invalids
            possible_fields[position] -= invalid_fields
            
            # shortlist confirmed fields with only one possibility
            if len(possible_fields[position]) == 1:
                remove_others(position, possible_fields)
    
    return possible_fields

def run_part2(input: List[str]):
    _, my_ticket, nearby_tickets = parse_all(input)
    fields = decode_fields(input)
    import math
    return math.prod([my_ticket[k] for k, v in fields.items()
              for field in v
              if field.startswith('departure')])

In [8]:
run_part2(real_ins)

2564529489989